In [5]:
import numpy as np
import pandas as pd
import time
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
#from skgarden import RandomForestQuantileRegressor          # pip install scikit-garden
import lightgbm as lgb                                       # pip install lightgbm
from sklearn.linear_model import QuantileRegressor           # sklearn ≥ 1.1

In [6]:
def winkler_score_and_coverage(df, obs, CIs):
    scores, obs = {}, np.asarray(obs)
    for ci in CIs:
        ci /= 100.0
        lq, uq = df.quantile((1-ci)/2), df.quantile(1-(1-ci)/2)
        inv_alpha = 1/(1-ci)
        lo, hi = obs <= lq, obs >= uq
        md = ~(lo | hi)
        w = np.zeros_like(obs, dtype=float)
        w[lo] = (uq[lo]-lq[lo]) + 2*inv_alpha*(lq[lo]-obs[lo])
        w[md] = (uq[md]-lq[md])
        w[hi] = (uq[hi]-lq[hi]) + 2*inv_alpha*(obs[hi]-uq[hi])
        scores[int(ci*100)] = {
            "winkler_scores": w.mean(),
            "coverage_probability": md.mean()
        }
    return pd.DataFrame(scores).T

In [7]:
train_data = pd.read_csv("train_2021.xls")
test_data = pd.read_csv("test_2022.xls")

In [8]:
FEATS = [
    'hour','minute','quarter','holiday','temp',
    'power_lag_1_day','power_lag_2_day','power_lag_3_day','power_lag_4_day'
]
X_train, y_train = train_data[FEATS], train_data['total_power']
X_test , y_test  = test_data [FEATS], test_data ['total_power']

x_scaler  = MinMaxScaler()
X_train_s = x_scaler.fit_transform(X_train)
X_test_s  = x_scaler.transform(X_test)

In [9]:
# Split training indices into 365 daily blocks (96 rows each)
n_rows_per_day = 96
train_day_idx = [
    np.arange(i*n_rows_per_day, (i+1)*n_rows_per_day)
    for i in range(len(y_train) // n_rows_per_day)
]

In [10]:
x_scaler = MinMaxScaler()
X_train_s = x_scaler.fit_transform(X_train)
X_test_s  = x_scaler.transform(X_test)

In [11]:
Q = [0.005,0.025,0.05,0.075,0.925,0.95,0.975,0.995]

In [12]:
preds = {m:{q:None for q in Q} for m in ["GBR","LGBM","LinQR"]}

In [9]:
for q in Q:
    start_time = time.time()
    gbr = GradientBoostingRegressor(
        loss="quantile", alpha=q,
        n_estimators=400, learning_rate=0.03,
        max_depth=3, random_state=0
    ).fit(X_train_s, y_train)
    preds["GBR"][q] = gbr.predict(X_test_s)
    print(time.time()-start_time)

25.42950963973999
24.935993194580078
24.991601705551147
25.15989327430725
24.787505626678467
24.766335487365723
24.859593629837036
25.058944940567017


In [4]:
for q in Q:
    start_time = time.time()
    lgb_q = lgb.LGBMRegressor(
        objective="quantile", alpha=q,
        n_estimators=1000, learning_rate=0.05,
        max_depth=-1, num_leaves=31, random_state=0, verbosity=-1
    ).fit(X_train_s, y_train)
    preds["LGBM"][q] = lgb_q.predict(X_test_s)
    print(time.time()-start_time)

41.65773153305054
27.782368659973145
22.20200777053833
17.51373529434204
17.525792121887207
15.941641330718994
11.177890300750732
8.218545913696289


In [13]:
for q in Q:
    start_time = time.time()
    lqr = QuantileRegressor(
        quantile=q, alpha=0.1, solver="highs"
    ).fit(X_train_s, y_train)
    preds["LinQR"][q] = lqr.predict(X_test_s)
    print(time.time()-start_time)
    

39.54107737541199
40.42747092247009
41.07562446594238
41.35956525802612
42.421167612075806
41.98199987411499
41.556337118148804
40.55710196495056


In [12]:
CI_LEVELS = [85,90,95,99]
q_bounds  = {85:(0.075,0.925), 90:(0.05,0.95),
             95:(0.025,0.975), 99:(0.005,0.995)}

for mdl in preds:
    start_time = time.time()
    # build DataFrame: 2×n_test for each CI
    tables = {}
    for cov,(ql,qh) in q_bounds.items():
        df_ci = pd.DataFrame(
            [preds[mdl][ql], preds[mdl][qh]],
            index=["lower","upper"]
        )
        tables[cov] = winkler_score_and_coverage(df_ci, y_test.values, [cov])
    print(time.time()-start_time)
    res_df = pd.concat(tables)
    res_df.to_csv(f"{mdl}_quantile_winkler.csv")
    print(f"\n=== {mdl}  Winkler / Coverage ===")
    display(res_df)

0.23530292510986328

=== GBR  Winkler / Coverage ===


,,coverage_probability,winkler_scores
85,85,0.743287,7.419024
90,90,0.829977,8.040592
95,95,0.916435,9.132515
99,99,0.974537,12.547331


0.16735172271728516

=== LGBM  Winkler / Coverage ===


,,coverage_probability,winkler_scores
85,85,0.683796,7.704420
90,90,0.781829,8.431987
95,95,0.869329,10.051803
99,99,0.950231,15.497496


0.16640901565551758

=== LinQR  Winkler / Coverage ===


,,coverage_probability,winkler_scores
85,85,0.741782,8.461609
90,90,0.826505,9.204869
95,95,0.905324,10.505656
99,99,0.974884,13.400220


In [13]:
preds = {m:{q:None for q in Q} for m in ["QRF"]}

In [14]:
from sklearn.ensemble import RandomForestRegressor

rf_plain = RandomForestRegressor(
    n_estimators=600,
    min_samples_leaf=1,
    random_state=0,
    n_jobs=-1
).fit(X_train_s, y_train)

# collect per-tree predictions → shape (n_trees, n_test)
tree_preds = np.stack([
    t.predict(X_test_s) for t in rf_plain.estimators_
])

for q in Q:                         # Q = [0.005, 0.025, …, 0.995]
    preds["QRF"][q] = np.percentile(tree_preds, q*100, axis=0)

In [15]:
bounds = {                       # coverage → (lower-q, upper-q)
    85 : (0.075, 0.925),
    90 : (0.05 , 0.95 ),
    95 : (0.025, 0.975),
    99 : (0.005, 0.995)
}

# ---- 3.  assemble a 2×n_test DataFrame per CI and score ----------
ci_tables = {}
for cov,(ql,qh) in bounds.items():
    df_ci = pd.DataFrame(
        [preds["QRF"][ql], preds["QRF"][qh]],   # rows: lower, upper
        index=["lower","upper"]
    )
    ci_tables[cov] = winkler_score_and_coverage(df_ci, y_test.values, [cov])

# ---- 4.  concatenate, display, save ------------------------------
qrf_tree_winkler = pd.concat(ci_tables)   # rows indexed by coverage level
print("\n=== Tree-percentile RF  —  Winkler & Coverage ===")
display(qrf_tree_winkler)

qrf_tree_winkler.to_csv("QRF_tree_winkler.csv")



=== Tree-percentile RF  —  Winkler & Coverage ===


,,coverage_probability,winkler_scores
85,85,0.752199,7.616849
90,90,0.830671,8.328592
95,95,0.910185,9.546197
99,99,0.971296,12.966486
